In [1]:
import json
import pickle
import random

import torch
from torch import nn, optim
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch.nn.utils.rnn as rnn_utils

import nltk
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
import time
import copy

from Vocab import Vocab
from LanguageModel import LanguageModel

import torch
torch.cuda.set_device(1)

print('import over')

copy_thres=1

import over


In [2]:
def batch_words2sentence(words_list):
    return [' '.join(words) for words in words_list]
def batch_tokens2words(tokens_list, vocab):
    ##    para: tokens_list is list[list] type
    ##    return: words_list corresponding to tokens
    return [[vocab.token2word[token] for token in tokens] for tokens in tokens_list]

def batch_tokens_remove_eos(tokens_list, vocab):
    ##    para: tokens_list is list[list] type
    ##    return pure tokens_list removed eos symbol
    result=[]
    for tokens in tokens_list:
        tokens_filtered=[]
        for token in tokens:
            if token == vocab.word2token['<eos>']:
#                 tokens_filtered.append(token)
                break
            else:
                tokens_filtered.append(token)
        result.append(tokens_filtered)
    return result

def batch_tokens_bleu(references, candidates, smooth_epsilon=0.001):
    ##    para: references and candidates are list[list] type
    ##    return: list of BLEU for every sample
    ##
    bleu_scores=[]
    for ref, candidate in zip(references, candidates):
        if min(len(ref), len(candidate))<4:
            bleu_scores.append(0)
        else:
            bleu_scores.append(sentence_bleu([ref], candidate, smoothing_function = SmoothingFunction(epsilon=smooth_epsilon).method1))
    return bleu_scores

with open('data_set/vocab.pk', 'rb') as f:
    vocab=pickle.load(f)

    
def seqs_split(seqs, vocab):
    seqs = batch_tokens_remove_eos(seqs, vocab)
    simple_sent1s=[]
    simple_sent2s=[]
    for seq in seqs:
        simple_sent1=[]
        simple_sent2=[]
        sent=simple_sent1
        for token in seq:
            if token==vocab.word2token['<split>']:
                sent=simple_sent2
            else:
                sent.append(token)
        simple_sent1s.append(simple_sent1)
        simple_sent2s.append(simple_sent2)
        
    return simple_sent1s, simple_sent2s

def simple_sents_concat(simple_sent1s, simple_sent2s, vocab, max_length):
    simple_sent_lens=[]
    simple_sents=simple_sent1s
    for i, sent in enumerate(simple_sent2s):
        simple_sents[i].append(vocab.word2token['<split>'])
        for token in sent:
            simple_sents[i].append(token)

        #if there is no <split> in simple_sent1s and simple_sent2s, then the length of sents_concat will be longer than max_length
        if len(simple_sents[i])>max_length:
            simple_sents[i] = simple_sents[i][:max_length]
            
        simple_sent_lens.append(len(simple_sents[i]))
            
        while(len(simple_sents[i])<max_length):
            simple_sents[i].append(vocab.word2token['<padding>'])
            
    return simple_sents, simple_sent_lens


def get_lm_inputs_and_labels(sents, vocab, max_length):
    lm_inputs=copy.deepcopy(sents)
    lm_labels=copy.deepcopy(sents)
    lm_input_lens=[]
    
    for sent in lm_inputs:
        if len(sent)>=max_length:
            sent=sent[:max_length-1]
        sent.insert(0, vocab.word2token['<sos>'])
        lm_input_lens.append(len(sent))
        while(len(sent)<max_length):
            sent.append(vocab.word2token['<padding>'])

    for sent in lm_labels:
        if len(sent)>=max_length:
            sent = sent[:max_length-1]
        sent.append(vocab.word2token['<eos>'])
        while(len(sent)<max_length):
            sent.append(vocab.word2token['<padding>'])
        
    return lm_inputs, lm_input_lens, lm_labels


def duplicate_reconstruct_labels(sents, topk):
    return [x for x in sents for ii in range(topk)]


def batch_tokens_bleu_split_version(references, candidates, vocab, smooth_epsilon=0.001):
    # needn't remove '<sos>' token before calling this function, which is different from the 'batch_token_bleu()' version
    #
    ref1, ref2 = seqs_split(references, vocab)
    cand1, cand2 = seqs_split(candidates, vocab)
    bleu_simple_sent1s = batch_tokens_bleu(ref1, cand1)
    bleu_simple_sent2s = batch_tokens_bleu(ref2, cand2)
#     print(bleu_simple_sent1s)
#     print(bleu_simple_sent2s)
    bleu=[]
    for idx in range(len(bleu_simple_sent1s)):
        bleu.append((bleu_simple_sent1s[idx]+bleu_simple_sent2s[idx])/2)
    return bleu


def set_model_grad(model, is_grad):
    for param in model.parameters():
         param.requires_grad = is_grad

In [3]:
seqs=[[8,9,90,5,3,2,1], [5,8,9,90,5,3,2,1], [8,2,9,40,5,3,2,2,1], [8,9,90,5,3,2,1], [8,9,90]]
a,b = seqs_split(seqs, vocab)

print(a)
print(b)

lm_in, lm_in_lens, lm_labels=get_lm_inputs_and_labels(a,vocab, max_length=6)
print(lm_in)
print(lm_in_lens)
print(lm_labels)
lm_in, lm_in_lens, lm_labels=get_lm_inputs_and_labels(b,vocab, max_length=6)
print(lm_in)
print(lm_in_lens)
print(lm_labels)

c,d=simple_sents_concat(a,b,vocab, 3)
print(c)
print(d)


batch_tokens_bleu([[1,2,3,4,5,6]], [[2,3,1,4,5]])

[[8, 9, 90], [], [8], [8, 9, 90], [8, 9, 90]]
[[3], [8, 9, 90, 3], [], [3], []]
[[0, 8, 9, 90, 1, 1], [0, 1, 1, 1, 1, 1], [0, 8, 1, 1, 1, 1], [0, 8, 9, 90, 1, 1], [0, 8, 9, 90, 1, 1]]
[4, 1, 2, 4, 4]
[[8, 9, 90, 2, 1, 1], [2, 1, 1, 1, 1, 1], [8, 2, 1, 1, 1, 1], [8, 9, 90, 2, 1, 1], [8, 9, 90, 2, 1, 1]]
[[0, 3, 1, 1, 1, 1], [0, 8, 9, 90, 3, 1], [0, 1, 1, 1, 1, 1], [0, 3, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1]]
[2, 5, 1, 2, 1]
[[3, 2, 1, 1, 1, 1], [8, 9, 90, 3, 2, 1], [2, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1]]
[[8, 9, 90], [5, 8, 9], [8, 5, 1], [8, 9, 90], [8, 9, 90]]
[3, 3, 2, 3, 3]


[0.013910597740964967]

In [4]:
#fusion data set

# with open('./data_set2/fusion_data_set/train_pseudo_simple_sents.pk', 'rb') as f:
#     fusion_pseudo_train_set_inputs = pickle.load(f)
# with open('./data_set2/fusion_data_set/train_pseudo_simple_sent_lens.pk', 'rb') as f:
#     fusion_pseudo_train_set_input_lens = pickle.load(f)
# with open('./data_set2/fusion_data_set/train_pseudo_labels.pk', 'rb') as f:
#     fusion_pseudo_train_set_labels = pickle.load(f)
#supervise
with open('./data_set2/fusion_data_set/train_simple_sents_supervised.pk', 'rb') as f:
    fusion_pseudo_train_set_inputs = pickle.load(f)
with open('./data_set2/fusion_data_set/train_simple_sent_lens_supervised.pk', 'rb') as f:
    fusion_pseudo_train_set_input_lens = pickle.load(f)
with open('./data_set2/fusion_data_set/train_labels_supervised.pk', 'rb') as f:
    fusion_pseudo_train_set_labels = pickle.load(f)
    
    
with open('./data_set2/fusion_data_set/validation_simple_sents.pk', 'rb') as f:
    fusion_pseudo_valid_set_inputs = pickle.load(f)
with open('./data_set2/fusion_data_set/validation_simple_sent_lens.pk', 'rb') as f:
    fusion_pseudo_valid_set_input_lens = pickle.load(f)
with open('./data_set2/fusion_data_set/validation_labels.pk', 'rb') as f:
    fusion_pseudo_valid_set_labels = pickle.load(f)
    
    
    
#split data set

# with open('./data_set2/split_data_set/train_complex_sents.pk', 'rb') as f:
#     split_train_set_inputs = pickle.load(f)
# with open('./data_set2/split_data_set/train_complex_sent_lens.pk', 'rb') as f:
#     split_train_set_input_lens = pickle.load(f)
# with open('./data_set2/split_data_set/train_pseudo_labels.pk', 'rb') as f:
#     split_pseudo_train_set_labels = pickle.load(f)
#supervise
with open('./data_set2/split_data_set/train_complex_sents_supervised.pk', 'rb') as f:
    split_train_set_inputs = pickle.load(f)
with open('./data_set2/split_data_set/train_complex_sent_lens_supervised.pk', 'rb') as f:
    split_train_set_input_lens = pickle.load(f)
with open('./data_set2/split_data_set/train_labels_supervised.pk', 'rb') as f:
    split_pseudo_train_set_labels = pickle.load(f)
    
    
with open('./data_set2/split_data_set/validation_complex_sents.pk', 'rb') as f:
    split_valid_set_inputs = pickle.load(f)
with open('./data_set2/split_data_set/validation_complex_sent_lens.pk', 'rb') as f:
    split_valid_set_input_lens = pickle.load(f)
with open('./data_set2/split_data_set/validation_labels.pk', 'rb') as f:
    split_pseudo_valid_set_labels = pickle.load(f)


In [5]:
print(len(split_train_set_inputs), len(split_train_set_input_lens), len(split_pseudo_train_set_labels))
print(len(fusion_pseudo_train_set_inputs), len(fusion_pseudo_train_set_input_lens), len(fusion_pseudo_train_set_labels))


197988 197988 197988
197988 197988 197988


In [6]:
class Encoder(nn.Module):
    def __init__(self, use_cuda, hidden_dim, input_dim, vocab):#, pre_train_weight, is_fix_word_vector = 1):
        super(Encoder, self).__init__()
        
        self.use_cuda = use_cuda
        self.input_dim=input_dim
        self.hidden_dim=hidden_dim
        self.vocab = vocab
        
        self.lstm=torch.nn.LSTM(input_size=self.input_dim, 
                                hidden_size= self.hidden_dim, 
                                bidirectional=True,
                                batch_first=True
                               )
        
        #embedding
        self.embed=nn.Embedding(len(self.vocab.word2token), input_dim)
        #loading pre trained word embedding
        with open('data_set/pre_trained_token_embedding.pk', 'rb') as f:
            pre_train_word_embedding = pickle.load(f)
            
        self.embed.weight.data.copy_(torch.FloatTensor(pre_train_word_embedding))
#         self.embed.weight.requires_grad = False
        
    def order(self, inputs, inputs_len):    #inputs: tensor, inputs_len: 1D tensor
        inputs_len, sort_ids = torch.sort(inputs_len, dim=0, descending=True)
        
        if self.use_cuda:
            inputs = inputs.index_select(0, Variable(sort_ids).cuda())
        else:
            inputs = inputs.index_select(0, Variable(sort_ids))
        
        _, true_order_ids = torch.sort(sort_ids, dim=0, descending=False)
        
        return inputs, inputs_len, true_order_ids
    #
    def forward(self, inputs, inputs_len):
        inputs = Variable(inputs)
        if self.use_cuda:
            inputs=inputs.cuda()
            
        inputs, sort_len, true_order_ids = self.order(inputs, inputs_len)

        in_vecs=self.embed(inputs)

        packed = rnn_utils.pack_padded_sequence(input=in_vecs, lengths=list(sort_len), batch_first =True)
        
        outputs, (hn,cn) = self.lstm(packed)
        outputs, sent_lens = rnn_utils.pad_packed_sequence(outputs)
        
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        outputs = outputs.transpose(0,1)  #transpose is necessary
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        
        #warnning: outputs, hn and cn have been sorted by sentences length so the order is wrong, now to sort them
        if self.use_cuda:
            outputs = outputs.index_select(0, Variable(true_order_ids).cuda())
        else:
            outputs = outputs.index_select(0, Variable(true_order_ids))
        
        hn = torch.cat((hn[0], hn[1]), dim=1)
        cn = torch.cat((cn[0], cn[1]), dim=1)
        #print('hn size and cn size: ', hn.size(), cn.size())
        
        if self.use_cuda:
            hn = hn.index_select(0, Variable(true_order_ids).cuda())
            cn = cn.index_select(0, Variable(true_order_ids).cuda())
        else:
            hn = hn.index_select(0, Variable(true_order_ids))
            cn = cn.index_select(0, Variable(true_order_ids))
            
        return outputs, (hn,cn)

In [7]:
def _inflate(tensor, times, dim):
    """
    Examples::
        >> a = torch.LongTensor([[1, 2], [3, 4]])
        >> a
        1   2
        3   4
        [torch.LongTensor of size 2x2]
        >> b = ._inflate(a, 2, dim=1)
        >> b
        1   2   1   2
        3   4   3   4
        [torch.LongTensor of size 2x4]
    """
    repeat_dims = [1] * tensor.dim()
    repeat_dims[dim] = times
    return tensor.repeat(*repeat_dims)

class Decoder(nn.Module):
    def __init__(self, use_cuda, encoder, hidden_dim, max_length=25):
        super(Decoder, self).__init__()
        
        self.use_cuda = use_cuda
        self.hidden_dim=hidden_dim
        self.input_dim = encoder.input_dim
        self.max_length = max_length
        self.vocab = encoder.vocab
        self.weight = [1]*len(self.vocab.word2token)
        self.weight[self.vocab.word2token['<padding>']]=0
        #self.weight[self.vocab.word2token['<eos>']]=1.01
        #self.weight[self.vocab.word2token['<split>']]=1.01
        
        self.hidden_size = self.hidden_dim
        self.V = len(self.vocab.word2token)
        self.SOS = self.vocab.word2token['<sos>']
        self.EOS = self.vocab.word2token['<eos>']
        self.log_softmax = nn.LogSoftmax(dim=1)
        
        self.lstmcell = torch.nn.LSTMCell(input_size=self.input_dim, hidden_size=self.hidden_dim*2, bias=True)
        
        #embedding
        self.embed=encoder.embed# reference share
        #fcnn: projection for crossentroy loss
        self.fcnn = nn.Linear(in_features = self.hidden_dim*2+hidden_dim*2, out_features = len(self.vocab.word2token))
        
        self.softmax = nn.Softmax(dim=1)
        self.cost_func = nn.CrossEntropyLoss(weight=torch.Tensor(self.weight), reduce=False)
        self.nll_loss = nn.NLLLoss(weight=torch.Tensor(self.weight), reduce=False)

        print('init lookup embedding matrix size: ', self.embed.weight.data.size())
        
        #copy
        out_features_dim=self.hidden_dim
        self.attent_wh = nn.Linear(in_features = self.hidden_dim*2, out_features = out_features_dim, bias = 0)
        self.attent_ws = nn.Linear(in_features = self.hidden_dim*2, out_features = out_features_dim, bias = 1)
        self.tanh = nn.Tanh()
        self.attent_vt = nn.Linear(in_features = out_features_dim, out_features = 1, bias=0)
        
        self.prob_wh = nn.Linear(in_features = self.hidden_dim*2, out_features = 1, bias=0)
        self.prob_ws = nn.Linear(in_features = self.hidden_dim*2, out_features = 1, bias=0)
        self.prob_wx = nn.Linear(in_features = input_dim, out_features = 1, bias=1)
        self.sigmoid = nn.Sigmoid()
    
    def get_context_vec(self, enc_outputs, this_timestep_input, dec_state):
        batch_size = enc_outputs.size(dim = 0)
        
        wh = self.attent_wh(enc_outputs)
        ws = self.attent_ws(dec_state).unsqueeze(dim=1)
#         print('wh, ws size: ', wh.size(), ws.size())
        ws = ws.expand(ws.size(0), wh.size(1), ws.size(2))
#         print('ws size: ', ws.size())
        weight = self.attent_vt(self.tanh(wh+ws))
#         print('weight size: ', weight.size())
        weight = self.softmax(weight.squeeze(dim=2))
#         print('weight size: ', weight.size())
        context_v = torch.bmm(weight.unsqueeze(dim=1), enc_outputs)
#         print('context_v size: ', context_v.size())
        context_v = context_v.squeeze(dim=1)
        return context_v, weight
    
    def copy_mechanism(self, enc_outputs, this_timestep_input, dec_state, inputs_one_hot, context_v, weight):
        batch_size = enc_outputs.size(dim = 0)
        
#         wh = self.attent_wh(enc_outputs)
#         ws = self.attent_ws(dec_state).unsqueeze(dim=1)
# #         print('wh, ws size: ', wh.size(), ws.size())
#         ws = ws.expand(ws.size(0), wh.size(1), ws.size(2))
# #         print('ws size: ', ws.size())
#         weight = self.attent_vt(self.tanh(wh+ws))
# #         print('weight size: ', weight.size())
#         weight = self.softmax(weight.squeeze(dim=2))
# #         print('weight size: ', weight.size())
#         context_v = torch.bmm(weight.unsqueeze(dim=1), enc_outputs)
# #         print('context_v size: ', context_v.size())
#         context_v = context_v.squeeze(dim=1)
        
        p_wh = self.prob_wh(context_v)
        p_ws = self.prob_ws(dec_state)
        p_wx = self.prob_wx(this_timestep_input)
        if_copy = self.sigmoid(p_wh+p_ws+p_wx)
#         if_copy = 0.3*if_copy
#         if_copy = self._tocuda(Variable(torch.ones(batch_size, 1), requires_grad=0))
#         print('if_copy size: ', if_copy.size())
        
        prob_copy = torch.bmm(inputs_one_hot, weight.unsqueeze(dim=2))
        prob_copy = prob_copy.squeeze(dim=2)
#         prob_copy = self._tocuda(Variable(torch.rand(batch_size, len(self.vocab.word2token)), requires_grad=0))
#         prob_copy = self.softmax(prob_copy)

#         print('prob_copy size: ', prob_copy.size())
#         print(torch.sum(prob_copy, dim=1))
#         print(torch.mean(if_copy))
        
#         if random.random()<0.005:
#             print('if_copy mean: ', torch.mean(if_copy))
#             _, max_ids = torch.max(prob_copy, dim=1)
#             print(self.vocab.token2word[max_ids.data[0]], self.vocab.token2word[max_ids.data[1]], self.vocab.token2word[max_ids.data[2]])
            
            
        return if_copy, prob_copy

    def forward(self, enc_outputs, sent_lens, h0_and_c0, labels, inputs, teaching_rate=0.6, is_train=1):
        labels = Variable(labels)
        if self.use_cuda:
            labels = labels.cuda()

        all_loss = 0
        predicts = []
        max_probs=[]
        batch_size = enc_outputs.size(dim = 0)
        final_hidden_states = h0_and_c0[0]
#         print('enc_outputs size:', enc_outputs.size())

        sents_len = enc_outputs.size(1)
        inputs = inputs[:,:sents_len].unsqueeze(dim=2)
        one_hot = torch.FloatTensor(batch_size, sents_len, len(self.vocab.word2token)).zero_()
        one_hot.scatter_(2, inputs, 1)
        one_hot = one_hot.transpose(1,2)
        one_hot = self._tocuda(Variable(one_hot, requires_grad = 0))
#         print('one_hot size: ', one_hot.size())
        
        for ii in range(self.max_length):
            if ii==0:
                zero_timestep_input = Variable(torch.LongTensor([self.vocab.word2token['<sos>']]*batch_size))
                if self.use_cuda:
                    zero_timestep_input = zero_timestep_input.cuda()
                    
                zero_timestep_input = self.embed(zero_timestep_input)#size: batch_size * self.input_dim
                
                last_timestep_hidden_state,cx = self.lstmcell(zero_timestep_input, h0_and_c0)
                #print('last_timestep_hidden_state: ', last_timestep_hidden_state.size(), cx.size())

                #get context vector
                context_vec, weight = self.get_context_vec(enc_outputs=enc_outputs, this_timestep_input=-1, 
                                                            dec_state = last_timestep_hidden_state)
                logits = self.fcnn(torch.cat([last_timestep_hidden_state, context_vec], dim=1))
                
                #copy or not
                copy_control=random.random()
                if copy_control<copy_thres:
                    if_copy, prob_copy = self.copy_mechanism(enc_outputs=enc_outputs, this_timestep_input=zero_timestep_input, 
                                                            dec_state = last_timestep_hidden_state, inputs_one_hot = one_hot, 
                                                            context_v=context_vec,
                                                            weight = weight)
                    score = (1-if_copy)*self.softmax(logits)+if_copy*prob_copy
                    score = torch.clamp(score, min=10**(-30), max=1)

                #for saving time: no training, no loss calculating
                if is_train:
                    if copy_control<copy_thres:
                        loss = self.nll_loss(torch.log(score), labels[:,0])
                    else:
                        loss = self.cost_func(logits, labels[:,0])
                    all_loss+=loss
                
                #get predicts
                if copy_control<copy_thres:
                    _, max_idxs = torch.max(score, dim=1)
                else:
                    _, max_idxs = torch.max(logits, dim=1)
                predicts.append(torch.unsqueeze(max_idxs, dim=0))
                
                
            else:
                if is_train:
                    rand = random.random()
                    if rand<teaching_rate:
                        this_timestep_input = self.embed(labels[:,ii-1])#label teaching, lookup embedding
                    else:
                        this_timestep_input = self.embed(max_idxs)#last_timestep output, and then look up word embedding
                else:
                    this_timestep_input = self.embed(max_idxs)#last_timestep output, and then look up word embedding
                
                last_timestep_hidden_state,cx = self.lstmcell(this_timestep_input, (last_timestep_hidden_state,cx))
                
                #get context vector
                context_vec, weight = self.get_context_vec(enc_outputs=enc_outputs, this_timestep_input=this_timestep_input, 
                                                            dec_state = last_timestep_hidden_state)
                logits = self.fcnn(torch.cat([last_timestep_hidden_state, context_vec], dim=1))
                
                #copy or not
                copy_control=random.random()
                if copy_control<copy_thres:
                    if_copy, prob_copy = self.copy_mechanism(enc_outputs=enc_outputs, this_timestep_input=this_timestep_input, 
                                                            dec_state = last_timestep_hidden_state, inputs_one_hot = one_hot, 
                                                             context_v=context_vec,
                                                            weight = weight)
                    score = (1-if_copy)*self.softmax(logits)+if_copy*prob_copy
                    score = torch.clamp(score, min=10**(-30), max=1)

                #for saving time: no training, no loss calculating
                if is_train:
                    if copy_control<copy_thres:
                        loss = self.nll_loss(torch.log(score), labels[:,ii])
                    else:
                        loss = self.cost_func(logits, labels[:,ii])
                    all_loss+=loss
                
                #get predicts
                if copy_control<copy_thres:
                    _, max_idxs = torch.max(score, dim=1)
                else:
                    _, max_idxs = torch.max(logits, dim=1)
                predicts.append(torch.unsqueeze(max_idxs, dim=0))
                
        predicts = torch.cat(predicts, dim=0)
        predicts = torch.transpose(predicts, 0, 1)
    
        if is_train:  #training
#             all_loss = torch.cat(all_loss, dim=1)
#             all_loss = torch.mean(all_loss, dim=1)
#             loss = torch.mean(all_loss)
            loss = all_loss/self.max_length
    
            #print('loss size: ', loss.size())
            #torch.cuda.empty_cache()
            if self.use_cuda:
                return loss, predicts.data.cpu().tolist()
            else:
                return loss, predicts.data.tolist()
        else:   #testing
            if self.use_cuda:
                return predicts.data.cpu().tolist()
            else:
                return predicts.data.tolist()
#         if is_train:  #training
#             if self.use_cuda:
#                 return all_loss/(self.max_length+1), predicts.data.cpu().numpy()
#             else:
#                 return all_loss/(self.max_length+1), predicts.data.numpy()
#         else:   #testing
#             if self.use_cuda:
#                 return predicts.data.cpu().numpy()
#             else:
#                 return predicts.data.numpy()
    
    
    def decode_topk_seqs(self, encoder, inputs, input_lens, topk=3):
        enc_outputs, (enc_hn, enc_cn) = encoder(inputs, input_lens)
        batch_size = enc_outputs.size(dim = 0)
        
        #one hot of inputs
        sents_len = enc_outputs.size(1)
        inputs = inputs[:,:sents_len].unsqueeze(dim=2)
        one_hot = torch.FloatTensor(batch_size, sents_len, len(self.vocab.word2token)).zero_()
        one_hot.scatter_(2, inputs, 1)
        one_hot = one_hot.transpose(1,2)
        one_hot = self._tocuda(Variable(one_hot, requires_grad = 0))
        
        metadata = self.decode_by_beamsearch(encoder_hidden=(enc_hn, enc_cn), encoder_outputs=enc_outputs, inputs_one_hot=one_hot,topk = topk)
        results = metadata['topk_sequence']
        results =torch.cat(results, dim = 2)
        results=results.view(batch_size*topk, -1)
        if self.use_cuda:
            results = results.data.cpu().tolist()
        else:
            results = results.data.tolist()
#         results=batch_tokens_remove_eos(results, self.vocab)

#         labels = [x for x in labels for ii in range(topk)]
#         labels = batch_tokens_remove_eos(labels, self.vocab)
#         bleu_scores = batch_tokens_bleu(references=labels, candidates=results, smooth_epsilon=0.01)
        
#         bleu_scores = torch.FloatTensor(bleu_scores).view(batch_size, topk)
#         bleu_max, _ = torch.max(bleu_scores, dim=1)
        
#         bleu_mean = torch.mean(bleu_scores, dim=1).unsqueeze(dim=1)
#         bleu_scores = bleu_scores-bleu_mean
#         bleu_scores = bleu_scores.view(-1)
        
#         bleu_scores = self._tocuda(Variable(bleu_scores, requires_grad = 0))
#         log_probs = metadata['score']
#         log_probs = log_probs.view(batch_size*topk)
#         loss = -torch.dot(log_probs, bleu_scores)/batch_size/topk
#         return loss, results, torch.mean(bleu_mean.squeeze()), torch.mean(bleu_max)

        log_probs = metadata['score']
        log_probs = log_probs.view(batch_size*topk)
        
        return results, log_probs
        
        
        
    def _tocuda(self, var):
        if self.use_cuda:
            return var.cuda()
        else:
            return var
    def decode_by_beamsearch(self, encoder_hidden=None, encoder_outputs=None, inputs_one_hot=None, topk = 10):
        self.k = topk
        batch_size = encoder_outputs.size(dim=0)
        
        self.pos_index = self._tocuda(Variable(torch.LongTensor(range(batch_size)) * self.k).view(-1, 1))

        hidden = tuple([_inflate(h, self.k, 1).view(batch_size*self.k, -1) for h in encoder_hidden])
        #print('hidden0 size: (%s, %s)'%(hidden[0].size(), hidden[1].size()))

        encoder_outputs = _inflate(encoder_outputs, self.k, 1).view(batch_size*self.k, encoder_outputs.size(1), encoder_outputs.size(2))
        inputs_one_hot = _inflate(inputs_one_hot, self.k, 1).view(batch_size*self.k, inputs_one_hot.size(1), inputs_one_hot.size(2))
        
        # Initialize the scores; for the first step,
        # ignore the inflated copies to avoid duplicate entries in the top k
        sequence_scores = torch.Tensor(batch_size * self.k, 1)
        sequence_scores.fill_(-float('Inf'))
        sequence_scores.index_fill_(0, torch.LongTensor([i * self.k for i in range(0, batch_size)]), 0.0)
        sequence_scores = self._tocuda(Variable(sequence_scores))

        # Initialize the input vector
        input_var = self._tocuda(Variable(torch.LongTensor([self.SOS] * batch_size * self.k)))

        # Store decisions for backtracking
        stored_outputs = list()
        stored_scores = list()
        stored_predecessors = list()
        stored_emitted_symbols = list()
        stored_hidden = list()

        for ii in range(0, self.max_length):
            # Run the RNN one step forward
            #print('setp: %s'%ii)
            input_vec = self.embed(input_var)
            #print('input_var and input_vec size: ', input_var.size(), input_vec.size())
            hidden = self.lstmcell(input_vec, hidden)
            #print('hidden size: (%s, %s)'%(hidden[0].size(), hidden[1].size()))
            
            #log_softmax_output = self.log_softmax(self.fcnn(hidden[0]))
            
            logits = self.fcnn(hidden[0])
#             print('logits size', logits.size())
#             print(encoder_outputs.size())
#             print(input_vec.size())
#             print(hidden[0].size())
#             print(inputs_one_hot.size())
            if_copy, prob_copy = self.copy_mechanism(enc_outputs=encoder_outputs, this_timestep_input=input_vec.squeeze(dim=1), 
                                                            dec_state = hidden[0], inputs_one_hot = inputs_one_hot)
#             print('if_copy size', if_copy.size(), 'prob_copy size', prob_copy.size())
            
            score = (1-if_copy)*self.softmax(logits)+if_copy*prob_copy
            score = torch.clamp(score, min=10**(-30), max=1)
#             print('score size: ', score.size())

            # To get the full sequence scores for the new candidates, add the local scores for t_i to the predecessor scores for t_(i-1)
            sequence_scores = _inflate(sequence_scores, self.V, 1)
            sequence_scores += torch.log(score).squeeze(1)
            scores, candidates = sequence_scores.view(batch_size, -1).topk(self.k, dim=1)

            # Reshape input = (bk, 1) and sequence_scores = (bk, 1)
            input_var = (candidates % self.V).view(batch_size * self.k, 1)
            sequence_scores = scores.view(batch_size * self.k, 1)

            # Update fields for next timestep
            predecessors = (candidates / self.V + self.pos_index.expand_as(candidates)).view(batch_size * self.k, 1)
            if isinstance(hidden, tuple):
                hidden = tuple([h.index_select(0, predecessors.squeeze()) for h in hidden])
            else:
                hidden = hidden.index_select(0, predecessors.squeeze())

            # Update sequence scores and erase scores for end-of-sentence symbol so that they aren't expanded
            stored_scores.append(sequence_scores.clone())
            eos_indices = input_var.data.eq(self.EOS)
            if eos_indices.nonzero().dim() > 0:
                sequence_scores.data.masked_fill_(eos_indices, -float('inf'))

            # Cache results for backtracking
            stored_predecessors.append(predecessors)
            stored_emitted_symbols.append(input_var)
#             stored_hidden.append(hidden)

        # Do backtracking to return the optimal values
        output, h_t, h_n, s, l, p = self._backtrack(hidden,
                                                    stored_predecessors, stored_emitted_symbols,
                                                    stored_scores, batch_size, self.hidden_size)

        metadata = {}

        metadata['score'] = s
        metadata['topk_length'] = l
        metadata['topk_sequence'] = p
        metadata['length'] = [seq_len[0] for seq_len in l]
        metadata['sequence'] = [seq[0] for seq in p]
        
#         torch.cuda.empty_cache()
        
        return metadata

    def _backtrack(self, hidden, predecessors, symbols, scores, b, hidden_size):
        """Backtracks over batch to generate optimal k-sequences.

        Args:
            nw_output [(batch*k, vocab_size)] * sequence_length: A Tensor of outputs from network
            nw_hidden [(num_layers, batch*k, hidden_size)] * sequence_length: A Tensor of hidden states from network
            predecessors [(batch*k)] * sequence_length: A Tensor of predecessors
            symbols [(batch*k)] * sequence_length: A Tensor of predicted tokens
            scores [(batch*k)] * sequence_length: A Tensor containing sequence scores for every token t = [0, ... , seq_len - 1]
            b: Size of the batch
            hidden_size: Size of the hidden state

        Returns:
            output [(batch, k, vocab_size)] * sequence_length: A list of the output probabilities (p_n)
            from the last layer of the RNN, for every n = [0, ... , seq_len - 1]

            h_t [(batch, k, hidden_size)] * sequence_length: A list containing the output features (h_n)
            from the last layer of the RNN, for every n = [0, ... , seq_len - 1]

            h_n(batch, k, hidden_size): A Tensor containing the last hidden state for all top-k sequences.

            score [batch, k]: A list containing the final scores for all top-k sequences

            length [batch, k]: A list specifying the length of each sequence in the top-k candidates

            p (batch, k, sequence_len): A Tensor containing predicted sequence
        """

        lstm = isinstance(hidden, tuple)

        # initialize return variables given different types
        output = list()
        h_t = list()
        p = list()
        # Placeholder for last hidden state of top-k sequences.
        # If a (top-k) sequence ends early in decoding, `h_n` contains
        # its hidden state when it sees EOS.  Otherwise, `h_n` contains
        # the last hidden state of decoding.
        if lstm:
            state_size = hidden[0].size()
            h_n = tuple([torch.zeros(state_size), torch.zeros(state_size)])
        else:
            h_n = torch.zeros(nw_hidden[0].size())
        l = [[self.max_length] * self.k for _ in range(b)]  # Placeholder for lengths of top-k sequences
                                                                # Similar to `h_n`

        # the last step output of the beams are not sorted
        # thus they are sorted here
        sorted_score, sorted_idx = scores[-1].view(b, self.k).topk(self.k)
        # initialize the sequence scores with the sorted last step beam scores
        s = sorted_score.clone()

        batch_eos_found = [0] * b   # the number of EOS found
                                    # in the backward loop below for each batch

        t = self.max_length - 1
        # initialize the back pointer with the sorted order of the last step beams.
        # add self.pos_index for indexing variable with b*k as the first dimension.
        t_predecessors = (sorted_idx + self.pos_index.expand_as(sorted_idx)).view(b * self.k)
        while t >= 0:
            # Re-order the variables with the back pointer
            current_symbol = symbols[t].index_select(0, t_predecessors)
            # Re-order the back pointer of the previous step with the back pointer of
            # the current step
            t_predecessors = predecessors[t].index_select(0, t_predecessors).squeeze()

            # This tricky block handles dropped sequences that see EOS earlier.
            # The basic idea is summarized below:
            #
            #   Terms:
            #       Ended sequences = sequences that see EOS early and dropped
            #       Survived sequences = sequences in the last step of the beams
            #
            #       Although the ended sequences are dropped during decoding,
            #   their generated symbols and complete backtracking information are still
            #   in the backtracking variables.
            #   For each batch, everytime we see an EOS in the backtracking process,
            #       1. If there is survived sequences in the return variables, replace
            #       the one with the lowest survived sequence score with the new ended
            #       sequences
            #       2. Otherwise, replace the ended sequence with the lowest sequence
            #       score with the new ended sequence
            #
            eos_indices = symbols[t].data.squeeze(1).eq(self.EOS).nonzero()
            if eos_indices.dim() > 0:
                for i in range(eos_indices.size(0)-1, -1, -1):
                    # Indices of the EOS symbol for both variables
                    # with b*k as the first dimension, and b, k for
                    # the first two dimensions
                    idx = eos_indices[i]
                    b_idx = int(idx[0] / self.k)
                    # The indices of the replacing position
                    # according to the replacement strategy noted above
                    res_k_idx = self.k - (batch_eos_found[b_idx] % self.k) - 1
                    batch_eos_found[b_idx] += 1
                    res_idx = b_idx * self.k + res_k_idx

                    # Replace the old information in return variables
                    # with the new ended sequence information
                    t_predecessors[res_idx] = predecessors[t][idx[0]]

                    current_symbol[res_idx, :] = symbols[t][idx[0]]
                    s[b_idx, res_k_idx] = scores[t][idx[0]]
                    l[b_idx][res_k_idx] = t + 1

            # record the back tracked results
            p.append(current_symbol)
            t -= 1

        # Sort and re-order again as the added ended sequences may change
        # the order (very unlikely)
        s, re_sorted_idx = s.topk(self.k)
        for b_idx in range(b):
            l[b_idx] = [l[b_idx][k_idx.data[0]] for k_idx in re_sorted_idx[b_idx,:]]

        re_sorted_idx = (re_sorted_idx + self.pos_index.expand_as(re_sorted_idx)).view(b * self.k)

        # Reverse the sequences and re-order at the same time
        # It is reversed because the backtracking happens in reverse time order
#         output = [step.index_select(0, re_sorted_idx).view(b, self.k, -1) for step in reversed(output)]
        p = [step.index_select(0, re_sorted_idx).view(b, self.k, -1) for step in reversed(p)]
        #    --- fake output ---
        output = None
        #    --- fake ---
        return output, h_t, h_n, s, l, p

    def _mask_symbol_scores(self, score, idx, masking_score=-float('inf')):
            score[idx] = masking_score

    def _mask(self, tensor, idx, dim=0, masking_score=-float('inf')):
        if len(idx.size()) > 0:
            indices = idx[:, 0]
            tensor.index_fill_(dim, indices, masking_score)

In [8]:
class Seq2Seq(nn.Module):
    def __init__(self, use_cuda, input_dim, hidden_dim, vocab, max_length = 25):
        super(Seq2Seq, self).__init__()
        
        self.use_cuda = use_cuda
        self.enc = Encoder(use_cuda=use_cuda, hidden_dim=hidden_dim, input_dim=input_dim, vocab=vocab)
        self.dec = Decoder(use_cuda=use_cuda, encoder=self.enc, hidden_dim=hidden_dim, max_length=max_length)
        if use_cuda:
            self.enc = self.enc.cuda()
            self.dec = self.dec.cuda()
    def forward(self, inputs, input_lens, labels, is_train=1, teaching_rate=1):
        enc_outputs, (enc_hn, enc_cn) = self.enc(torch.LongTensor(inputs), torch.LongTensor(input_lens))
        if is_train:
            loss, predicts = self.dec(enc_outputs = enc_outputs, 
                                    h0_and_c0=(enc_hn, enc_cn), 
                                    sent_lens=input_lens,
                                    labels=torch.LongTensor(labels), 
                                    is_train=1, 
                                    teaching_rate = 1,
                                    inputs = inputs
                                    )
            return loss, predicts
        else:
            predicts = self.dec(enc_outputs = enc_outputs, 
                                h0_and_c0=(enc_hn, enc_cn), 
                                sent_lens=input_lens,
                                labels=torch.LongTensor(labels), 
                                is_train=0, 
                                teaching_rate = 1,
                                inputs = inputs
                                )
            return predicts
#     def train_using_rl(self, inputs, input_lens, labels, is_train=1, teaching_rate=1):
#         enc_outputs, (enc_hn, enc_cn) = self.enc(torch.LongTensor(inputs), torch.LongTensor(input_lens))
#         loss, predicts, bleu_mean = self.dec.train_using_rl_2(enc_outputs = enc_outputs, 
#                                                 h0_and_c0=(enc_hn, enc_cn), 
#                                                 sent_lens=input_lens,
#                                                 labels=labels,
#                                                 is_train=1, 
#                                                 teaching_rate = 1
#                                                 )
#         return loss, predicts, bleu_mean

    def tocuda(self, x):
        if self.use_cuda:
            return x.cuda()
        else:
            return x
        
    def train_using_reward(self, inputs, input_lens, reconstruct_labels, reconstruct_model, language_model, topk=3, loss_ratio=0.5):
        dec_seqs, log_probs = self.dec.decode_topk_seqs(self.enc, inputs, input_lens, topk=topk)
#         enc_outputs, (enc_hn, enc_cn) = self.enc(torch.LongTensor(inputs), torch.LongTensor(input_lens))
#         results = self.dec.decode_no_labels(enc_outputs=enc_outputs, h0_and_c0=(enc_hn, enc_cn), topk=topk)
        simple_sent1s, simple_sent2s = seqs_split(dec_seqs, self.enc.vocab)
        
        lm_input1s, lm_input1_lens, lm_label1s = get_lm_inputs_and_labels(simple_sent1s, self.enc.vocab, self.dec.max_length)
        simple_sent1s_ppl = language_model.get_sentences_ppl(torch.LongTensor(lm_input1s), 
                                                      torch.LongTensor(lm_input1_lens), 
                                                      torch.LongTensor(lm_label1s)
                                                    )
        lm_input2s, lm_input2_lens, lm_label2s = get_lm_inputs_and_labels(simple_sent2s, self.enc.vocab, self.dec.max_length)
        simple_sent2s_ppl = language_model.get_sentences_ppl(torch.LongTensor(lm_input2s), 
                                                      torch.LongTensor(lm_input2_lens), 
                                                      torch.LongTensor(lm_label2s)
                                                    )
        
        simple_inputs, simple_input_lens = simple_sents_concat(simple_sent1s, simple_sent2s, self.enc.vocab, self.dec.max_length)
        #reconstruct labels
        reconstruct_loss, predicts = reconstruct_model.forward(torch.LongTensor(simple_inputs), 
                                     torch.LongTensor(simple_input_lens), 
                                     labels=reconstruct_labels, 
                                     is_train=1, teaching_rate=1)
        
        #rm_rewards: reconstruct model rewards
        #lm_rewards: language model rewards
        rm_rewards=-reconstruct_loss.data
        lm_rewards=(1/self.tocuda(torch.Tensor(simple_sent1s_ppl))+1/self.tocuda(torch.Tensor(simple_sent2s_ppl)))/2
        
        rm_rewards_mean = torch.mean(rm_rewards.view(-1, topk), dim=1)
        lm_rewards_mean = torch.mean(lm_rewards.view(-1, topk), dim=1)
        rm_rewards = rm_rewards.view(-1, topk) - rm_rewards_mean.unsqueeze(dim=1)
        lm_rewards = lm_rewards.view(-1, topk) - lm_rewards_mean.unsqueeze(dim=1)
        
        rm_rewards = rm_rewards.view(-1)
        lm_rewards = lm_rewards.view(-1)
        
        #sum both rewards up
        rewards = loss_ratio*rm_rewards+(1-loss_ratio)*lm_rewards
        rewards = Variable(rewards, requires_grad=0)
        
        #regarding rewards as weights of every seq
        loss = -torch.dot(log_probs, rewards)/log_probs.size(0)
        
#         labels = [x for x in labels for ii in range(topk)]
#         labels = batch_tokens_remove_eos(labels, self.vocab)
#         bleu_scores = batch_tokens_bleu(references=labels, candidates=results, smooth_epsilon=0.01)
        
#         bleu_scores = torch.FloatTensor(bleu_scores).view(batch_size, topk)
#         bleu_max, _ = torch.max(bleu_scores, dim=1)
        
#         bleu_mean = torch.mean(bleu_scores, dim=1).unsqueeze(dim=1)
#         bleu_scores = bleu_scores-bleu_mean
#         bleu_scores = bleu_scores.view(-1)
        
#         bleu_scores = self._tocuda(Variable(bleu_scores, requires_grad = 0))
        
#         log_probs = metadata['score']
#         log_probs = log_probs.view(batch_size*topk)
    
#         loss = -torch.dot(log_probs, bleu_scores)/batch_size/topk
        
        return loss, reconstruct_loss, torch.mean(rm_rewards_mean), torch.mean(lm_rewards_mean)
    
    


In [ ]:
def split_model_eval(model, inputs, input_lens, labels):
    dataset_size = len(inputs)
    print(dataset_size)
    scores_ground_truth=0
    scores_no_ground_truth=0
    for idx in range(0, dataset_size, batch_size):
        
        #no teacher forcing
        predicts = model.forward(torch.LongTensor(inputs[idx:idx+batch_size]),
                                 torch.LongTensor(input_lens[idx:idx+batch_size]),
                                 labels=[],
                                 is_train=0, teaching_rate=1)
        bleu_scores = batch_tokens_bleu_split_version(references = labels[idx:idx+batch_size],
                                                     candidates = predicts,
                                                     smooth_epsilon=0.001,
                                                     vocab=vocab)
        for x in bleu_scores:
            scores_no_ground_truth+=x
    return scores_no_ground_truth/dataset_size


def fusion_model_eval(model, inputs, input_lens, labels):
    dataset_size = len(inputs)
    scores_ground_truth=0
    scores_no_ground_truth=0
    for idx in range(0, dataset_size, batch_size):
        
        #no teacher forcing
        predicts = model.forward(torch.LongTensor(inputs[idx:idx+batch_size]),
                                 torch.LongTensor(input_lens[idx:idx+batch_size]),
                                 labels=[],
                                 is_train=0, teaching_rate=1)
        predicts = batch_tokens_remove_eos(predicts, vocab)
        labels_ = batch_tokens_remove_eos(labels[idx:idx+batch_size], vocab)
        bleu_scores = batch_tokens_bleu(references=labels_, candidates=predicts, smooth_epsilon=0.001)
        for x in bleu_scores:
            scores_no_ground_truth+=x
    return scores_no_ground_truth/dataset_size



In [ ]:
use_cuda = 1
hidden_dim = 256
input_dim = 100
lr=0.005
batch_size=100
split_train_set_size=int(len(split_train_set_inputs)/2)
epochs=10000
train_bleu_mean=-1
train_bleu_max=-1
split_model = Seq2Seq(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, 
                          vocab = vocab, max_length = 61)

fusion_model = Seq2Seq(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, 
                          vocab = vocab, max_length = 51)
# #pre train para
# split_model_path = './models_saved/time-[2019-03-09-18-40-05]-info=[pre-trained_split_model-20per]-loss=0.673111856-bleu=0.7492-hidden_dim=256-input_dim=100-epoch=1-batch_size=100-batch_id=[1-[of]-1979]-lr=0.0050'
# fusion_model_path = './models_saved/time-[2019-03-09-18-40-07]-info=[pre-trained_fusion_model-20per]-loss=0.515186548-bleu=0.3736-hidden_dim=256-input_dim=100-epoch=1-batch_size=100-batch_id=[1-[of]-1979]-lr=0.0050'

# split_model_path = './models_saved/time-[2019-03-10-13-23-10]-info=[pre-trained_split_model-20per]-loss=0.454687029-bleu=0.7130-hidden_dim=256-input_dim=100-epoch=4-batch_size=100-batch_id=[1-[of]-1979]-lr=0.0050'
# fusion_model_path = './models_saved/time-[2019-03-10-13-23-11]-info=[pre-trained_fusion_model-20per]-loss=0.346116364-bleu=0.7466-hidden_dim=256-input_dim=100-epoch=4-batch_size=100-batch_id=[1-[of]-1979]-lr=0.0050'

# pre_train = torch.load(split_model_path, map_location='cpu')
# split_model.load_state_dict(pre_train)
# pre_train = torch.load(fusion_model_path, map_location='cpu')
# fusion_model.load_state_dict(pre_train)

if use_cuda:
    split_model = split_model.cuda()
    fusion_model = fusion_model.cuda()
    
split_optimizer = optim.Adam(filter(lambda p: p.requires_grad, split_model.parameters()), lr=lr)
fusion_optimizer = optim.Adam(filter(lambda p: p.requires_grad, fusion_model.parameters()), lr=lr)

start_time = time.time()

def model_train(epoch, batch_size, train_set_size):
    batch_id = 0
    valid_bleu = 0
    for start_idx in range(0, train_set_size-batch_size+1, batch_size):
#         print('batch id: ', batch_id)
            
        batch_id+=1
        end_idx = start_idx + batch_size
        
        split_optimizer.zero_grad()#clear  
        split_loss, predicts = split_model.forward(torch.LongTensor(split_train_set_inputs[start_idx:end_idx]), 
                                     torch.LongTensor(split_train_set_input_lens[start_idx:end_idx]), 
                                     labels=torch.LongTensor(split_pseudo_train_set_labels[start_idx:end_idx]), 
                                     is_train=1, teaching_rate=1)
        split_loss=torch.mean(split_loss)

        #optimize
        split_loss.backward()#retain_graph=True)
        split_optimizer.step()
        
#         torch.cuda.empty_cache()
        
        fusion_optimizer.zero_grad()#clear
        fusion_loss, predicts = fusion_model.forward(torch.LongTensor(fusion_pseudo_train_set_inputs[start_idx:end_idx]), 
                                     torch.LongTensor(fusion_pseudo_train_set_input_lens[start_idx:end_idx]), 
                                     labels=torch.LongTensor(fusion_pseudo_train_set_labels[start_idx:end_idx]), 
                                     is_train=1, teaching_rate=1)
        fusion_loss = torch.mean(fusion_loss)
        fusion_loss.backward()#retain_graph=True)
        fusion_optimizer.step()

#         torch.cuda.empty_cache()
        
        if batch_id%50==1:
            split_model.eval()
            fusion_model.eval()
            
            sample_num = 5
            rand_idx = random.randint(0, train_set_size-sample_num-1)
            
            print('--------split model training sampling display--------')
            #teaching forcing
            loss_, predicts = split_model.forward(torch.LongTensor(split_train_set_inputs[rand_idx:rand_idx+sample_num]), 
                                             torch.LongTensor(split_train_set_input_lens[rand_idx:rand_idx+sample_num]), 
                                             labels=torch.LongTensor(split_pseudo_train_set_labels[rand_idx:rand_idx+sample_num]), 
                                             is_train=1, teaching_rate=1)
            del loss_
            
            predicts = batch_tokens_remove_eos(predicts, vocab)
            labels = batch_tokens_remove_eos(split_pseudo_train_set_labels[rand_idx:rand_idx+sample_num], vocab)
            
            predicts = batch_tokens2words(predicts, vocab)
            labels = batch_tokens2words(labels, vocab)
            
            predicts_sents = batch_words2sentence(predicts)
            labels_sents = batch_words2sentence(labels)
            
            for (predict_sent, label_sent) in zip(predicts_sents, labels_sents):
                print(' 1----> ', predict_sent)
                print(' 2----> ', label_sent)
                print('\n')
                
            print('--------fusion model training sampling display--------')
            loss_, predicts = fusion_model.forward(torch.LongTensor(fusion_pseudo_train_set_inputs[rand_idx:rand_idx+sample_num]), 
                                             torch.LongTensor(fusion_pseudo_train_set_input_lens[rand_idx:rand_idx+sample_num]), 
                                             labels=torch.LongTensor(fusion_pseudo_train_set_labels[rand_idx:rand_idx+sample_num]), 
                                             is_train=1, teaching_rate=1)
            del loss_
            
            predicts = batch_tokens_remove_eos(predicts, vocab)
            labels = batch_tokens_remove_eos(fusion_pseudo_train_set_labels[rand_idx:rand_idx+sample_num], vocab)
            
            predicts = batch_tokens2words(predicts, vocab)
            labels = batch_tokens2words(labels, vocab)
            
            predicts_sents = batch_words2sentence(predicts)
            labels_sents = batch_words2sentence(labels)
            
            for (predict_sent, label_sent) in zip(predicts_sents, labels_sents):
                print(' 1----> ', predict_sent)
                print(' 2----> ', label_sent)
                print('\n')
                
#             #no teaching forcing
            print('----no teaching forcing----')
            predicts = split_model.forward(torch.LongTensor(fusion_pseudo_train_set_inputs[rand_idx:rand_idx+sample_num]), 
                                             torch.LongTensor(fusion_pseudo_train_set_input_lens[rand_idx:rand_idx+sample_num]), 
                                             labels=[],
                                             is_train=0, teaching_rate=1)
            predicts = batch_tokens_remove_eos(predicts, vocab)
            labels = batch_tokens_remove_eos(split_pseudo_train_set_labels[rand_idx:rand_idx+sample_num], vocab)
            
            predicts = batch_tokens2words(predicts, vocab)
            labels = batch_tokens2words(labels, vocab)
            
            predicts_sents = batch_words2sentence(predicts)
            labels_sents = batch_words2sentence(labels)

            for (predict_sent, label_sent) in zip(predicts_sents, labels_sents):
                print(' 1----> ', predict_sent)
                print(' 2----> ', label_sent)
                print('\n')
                
            info_stamp = 'split_loss={:2.9f}-train_bleu_mean={:2.9f}-train_bleu_max={:2.9f}-batch_size={:n}-epoch={:n}-batch_id=({:n}/{:n})'.format(
                              split_loss.data[0], train_bleu_mean, train_bleu_max, batch_size, epoch, batch_id, int(train_set_size/batch_size))
            print(info_stamp)
#             torch.cuda.empty_cache()
            
# #             #valid_set testing
            if batch_id%500==1:
#                 valid_bleu = split_model_eval(split_model, split_valid_set_inputs, split_valid_set_input_lens, split_pseudo_valid_set_labels)
                rand_idx=random.randint(0, len(split_valid_set_inputs)-batch_size-1-1)
                predicts = split_model.forward(torch.LongTensor(split_valid_set_inputs[rand_idx:rand_idx+batch_size]), 
                                                 torch.LongTensor(split_valid_set_input_lens[rand_idx:rand_idx+batch_size]), 
                                                 labels=[],#torch.LongTensor(valid_set_labels[rand_idx:rand_idx+batch_size]), 
                                                 is_train=0, teaching_rate=1)
#                 predicts = batch_tokens_remove_eos(predicts, vocab)
#                 labels = batch_tokens_remove_eos(split_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size], vocab)
                
#                 bleu_scores = batch_tokens_bleu(references=labels, candidates=predicts, smooth_epsilon=0.001)
                bleu_scores = batch_tokens_bleu_split_version(references = split_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size],
                                                             candidates = predicts,
                                                             smooth_epsilon=0.001,
                                                             vocab=vocab)
                valid_bleu = 0
                for x in bleu_scores:
                    valid_bleu+=x
                valid_bleu/=len(bleu_scores)
                       
                info_stamp = 'info=[{:s}]-loss={:2.9f}-bleu={:1.4f}-hidden_dim={:n}-input_dim={:n}-epoch={:n}-batch_size={:n}-batch_id=[{:n}-[of]-{:n}]-lr={:1.4f}'.format(
                              'pre-trained_split_model-10per', split_loss.data[0], valid_bleu, hidden_dim, input_dim, epoch, batch_size, batch_id, int(train_set_size/batch_size), lr)
                print(info_stamp, valid_bleu)
                now = int(round(time.time()*1000))
                time_stamp = time.strftime('time-[%Y-%m-%d-%H-%M-%S]-',time.localtime(now/1000))
                torch.save(split_model.state_dict(), ''.join(['./models_saved/', time_stamp, info_stamp]))
                
                #fusion eval
#                 valid_bleu = fusion_model_eval(fusion_model, fusion_pseudo_valid_set_inputs, 
#                                                   fusion_pseudo_valid_set_input_lens, fusion_pseudo_valid_set_labels)
                


                rand_idx=random.randint(0, len(fusion_pseudo_valid_set_inputs)-batch_size-1-1)
                predicts = fusion_model.forward(torch.LongTensor(fusion_pseudo_valid_set_inputs[rand_idx:rand_idx+batch_size]), 
                                                 torch.LongTensor(fusion_pseudo_valid_set_input_lens[rand_idx:rand_idx+batch_size]), 
                                                 labels=[],#torch.LongTensor(valid_set_labels[rand_idx:rand_idx+batch_size]), 
                                                 is_train=0, teaching_rate=1)
                predicts = batch_tokens_remove_eos(predicts, vocab)
                labels = batch_tokens_remove_eos(fusion_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size], vocab)
                
                bleu_scores = batch_tokens_bleu(references=labels, candidates=predicts, smooth_epsilon=0.001)

                valid_bleu = 0
                for x in bleu_scores:
                    valid_bleu+=x
                valid_bleu/=len(bleu_scores)
                
                info_stamp = 'info=[{:s}]-loss={:2.9f}-bleu={:1.4f}-hidden_dim={:n}-input_dim={:n}-epoch={:n}-batch_size={:n}-batch_id=[{:n}-[of]-{:n}]-lr={:1.4f}'.format(
                              'pre-trained_fusion_model-10per', fusion_loss.data[0], valid_bleu, hidden_dim, input_dim, epoch, batch_size, batch_id, int(train_set_size/batch_size), lr)
                print(info_stamp, valid_bleu)
                now = int(round(time.time()*1000))
                time_stamp = time.strftime('time-[%Y-%m-%d-%H-%M-%S]-',time.localtime(now/1000))
                torch.save(fusion_model.state_dict(), ''.join(['./models_saved/', time_stamp, info_stamp]))
#                 torch.cuda.empty_cache()
            split_model.train()
            fusion_model.train()
            
for epoch in range(epochs):
    model_train(epoch, batch_size, split_train_set_size)
    
print('running time: %.2f mins'%((time.time()-start_time)/60))

init lookup embedding matrix size:  torch.Size([44380, 100])
init lookup embedding matrix size:  torch.Size([44380, 100])
--------split model training sampling display--------
 1---->  in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in
 2---->  beginning in 1979 , bjork - <low_freq> valves with . <split> the <low_freq> - concave bjork - <low_freq> valve had a tendency to develop fractures in the outflow strut which could result in catastrophic valve failure and possibly sudden cardiac death .


 1---->  the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
 2---->  beginning in 1980 with the iowa department of education , 5,000 school districts around the world purchase

 1---->  <padding> in in general <low_freq> school school a a a a a a a a . . , , . forms . <split> this is , , , , forms forms forms forms forms
 2---->  in general <low_freq> school is a very impressive , with a small majority of <low_freq> and a high majority of <low_freq> . <split> it also has very skilled children , in many forms .


 1---->  <padding> in in <low_freq> <low_freq> 19 , , , , , , . , . . . <split> this , , , , , 's of ladder of ladder .
 2---->  in genesis <low_freq> - 19 , abraham 's grandson jacob left beersheba and went toward <low_freq> . <split> along the way he had his dream of jacob 's ladder .


 1---->  in in in geneva , his ministry both attracted other protestant protestant protestant and and and . his . . the the . the . . <split> this is the the the the of the his his his of his his . his his his .
 2---->  in geneva , his ministry both attracted other protestant refugees and over time made that city a major force in the spread of reformed theology . <s

 1---->  jacques washer , antiquarian who died in <low_freq> flight 316 and <split> . <split> the <split> of the <split> of the age of the six of six of six of six of six of six of six six six six six six six six six six six six six six six six six six six six six six six six six
 2---->  jacques washer , antiquarian who died in the <low_freq> flight 316 crash and edouard . <split> philippe started playing tennis at the age of six .


 1---->  jacques <low_freq> , was principal ballet dancer with new york city . <split> the <split> , the the the the starlight '' of the the the the film '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 2---->  jacques <low_freq> , was a principal ballet dancer with the new york city ballet . <split> he danced the role of the '' starlight carnival boy '' in the film '' carousel '' .


 1---->  <low_freq> <low_freq> jean frederica eliot , countess of germans ( 29 ) ) ) ) ) ) of wedderburn . <split> the <lo

--------split model training sampling display--------
 1---->  between 1971 and 1975 the organisation became carrier group 4 affiliated affiliated with striking fleet . atlantic . and . 2004 . <split> he 2004 , became carrier strike group 4 .
 2---->  between 1971 and 1975 the organisation became carrier group 4 , affiliated with nato striking fleet atlantic , part of <low_freq> . <split> in 2004 it became carrier strike group 4 .


 1---->  between 1971 and 1982 , roy <low_freq> served as deputy premier of saskatchewan . <split> from 1987 premier premier he , the to led . he 2001 . 2001 saskatchewan new democratic party .
 2---->  between 1971 and 1982 , roy <low_freq> served as deputy premier of saskatchewan . <split> on november 7 , 1987 mr. <low_freq> replaced allan <low_freq> as leader of the saskatchewan new democratic party .


 1---->  between 1971 and 1991 , mortality from heart disease dropped 41 percent , strokes decreased by 59 percent . <split> today today more than 70 per

 1---->  eastern parkway was a station on the demolished bmt fulton street <low_freq> had 2 tracks and 1 island platform was served served by trains of the bmt fulton street line .
 2---->  eastern parkway was a station on the demolished bmt fulton street <low_freq> had 2 tracks and 1 island platform and was served by trains of the bmt fulton street line .


 1---->  eastern slavonia is a mostly flat area , with the best type of soil where agriculture is highly developed , particularly on wheat fields . it has also has several forests as well as vineyards .
 2---->  eastern slavonia is a mostly flat area , with the best type of soil where agriculture is highly developed , particularly on wheat fields , and it also has several forests as well as vineyards .


 1---->  eastern twin ( <low_freq> ) and cinematic volunteer composed and produced by martin tillman and tom <low_freq> and the by by michael <low_freq> , cello solos performed by michael tillman , tim performed <low_freq> , perfor

...

 1---->  for that article , editor warren <low_freq> paired thompson with illustrator ralph steadman , who drew to expressionist expressionist illustrations with lipstick and <low_freq> .
 2---->  for that article , editor warren <low_freq> paired thompson with illustrator ralph steadman , who claimed to drew expressionist illustrations with lipstick and <low_freq> .


 1---->  for that reason , a is often used in conjunction with computational fluid dynamics tools , are better able to account for geometric complexity , at the expense of being more limited in their treatment of the underlying chemistry of the reactive process being studied .
 2---->  for that reason , <low_freq> is often used in conjunction with computational fluid dynamics tools that are better able to account for geometric complexity , at the expense of being more limited in their treatment of the underlying chemistry of the reactive process being studied .


 1---->  for that reason , game data such as levels , grap

 1---->  his father worked as a petition writer in district <low_freq> , <low_freq> . <split> he died in 1899 when master ji was only 12 years old .
 2---->  his father worked as a petition writer in district <low_freq> , <low_freq> . <split> he died in 1899 when master ji was only 12 years old .


 1---->  his father worked for a long time trying to turn him back to normal practice . <split> after much experimenting , he was finally freed thanks to static , gear and <low_freq> .
 2---->  his father worked for a long time trying to turn him back to normal . <split> after much experimenting , he was finally freed thanks to static , gear and <low_freq> .


 1---->  his father worked irregularly as a day laborer and indulged in gambling . <split> <split> john gotti came to resent his father for being unable to provide for his family .
 2---->  his father worked irregularly as a day laborer and indulged in gambling . <split> as an adult , john gotti came to resent his father for being unab

--------split model training sampling display--------
 1---->  in five years , global sales of highway legal light - duty plug - in electric vehicles increased increased more than <low_freq> , totaling more than <low_freq> units in 2015 . <split> itself - in electric were 2015 - more <low_freq> % increase 2014 , driven mainly by china and europe .
 2---->  in five years , global sales of highway legal light - duty plug - in electric vehicles have increased more than <low_freq> , totaling more than <low_freq> units in 2015 . <split> plug - in sales in 2015 increased about 80 % from 2014 , driven mainly by china and europe .


 1---->  in fixed - wing aircraft driven by one or more jet engines , the performance of the jet engine is important to the operation of the jet . <split> the includes the jet engine includes measurement of thrust , fuel consumption , noise and engine emissions .
 2---->  in fixed - wing aircraft driven by one or more jet engines , the performance of the jet engine

info=[pre-trained_fusion_model-10per]-loss=0.235383555-bleu=0.7049-hidden_dim=256-input_dim=100-epoch=8-batch_size=100-batch_id=[1-[of]-989]-lr=0.0050 0.704863605498269
--------split model training sampling display--------
 1---->  in 1984 newham council voted to demolish ronan point . <split> all nine blocks on the estate , comprising 990 flats , were demolished and the and the area was redeveloped with 20 two - storey houses with gardens .
 2---->  in 1984 newham council voted to demolish ronan point . <split> all nine blocks on the estate , comprising 990 flats , were demolished in 1986 and the area was redeveloped with 20 two - storey houses with gardens .


 1---->  in 1984 an eight room permanent a a - pak was added to the school to ease some of the overcrowding . <split> through the 1980 's several new schools were built in the area to further some of the school 's enrollment pressures .
 2---->  in 1984 an eight room port - a - pak was added to the school to ease some of the ov

 1---->  he was deputy inspector - general of police , <low_freq> range , joint commissioner of police , chennai city . <split> following matriculation , chennai was dig of police , tiruchirapalli before becoming chief vigilance officer , <low_freq> , at <low_freq> .
 2---->  he was deputy inspector - general of police , <low_freq> range , joint commissioner of police , chennai city . <split> later , he was dig of police , tiruchirapalli before becoming chief vigilance officer , <low_freq> , at <low_freq> .


 1---->  he was editor - in - chief of the <low_freq> <low_freq> editorial board . <split> he also has a member of several editorial boards .
 2---->  he was editor - in - chief of the <low_freq> <low_freq> editorial board and also member of several editorial boards . <split> he is currently at the national universities commission .


 1---->  he was executive secretary of the irish catholic bishops ' conference from 1991 to 1997 . <split> he published a book on '' the irish <low_

In [ ]:
stop

In [ ]:
lm_hidden_dim=512
lm_input_dim=300
use_cuda=1

language_model = LanguageModel(use_cuda = use_cuda, input_dim = lm_input_dim, hidden_dim = lm_hidden_dim, vocab = vocab)
#512
model_path = './models_language_model/time-[2019-02-26-13-18-56]-info=[language_model]-loss=4.003012180-bleu=-1.0000-hidden_dim=512-input_dim=300-epoch=24-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
#2048
# model_path = './models_language_model/time-[2019-02-28-07-04-08]-info=[language_model]-loss=3.475848675-bleu=-1.0000-hidden_dim=2048-input_dim=300-epoch=4-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
# #1024
# model_path = './models_language_model/time-[2019-02-27-21-58-23]-info=[language_model]-loss=4.111208439-bleu=-1.0000-hidden_dim=1024-input_dim=300-epoch=6-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'

pre_train = torch.load(model_path, map_location='cpu')
language_model.load_state_dict(pre_train)

if use_cuda:
    language_model = language_model.cuda()
    
language_model.eval()

print('finish loading pre-train weight for language model.')



use_cuda = 1
hidden_dim = 256
input_dim = 100
lr=0.005

split_model = Seq2Seq(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, 
                          vocab = vocab, max_length = 61)

fusion_model = Seq2Seq(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, 
                          vocab = vocab, max_length = 51)
#pre train para
split_model_path = './models_saved/time-[2019-03-10-01-36-10]-info=[pre-trained_split_model-20per]-loss=0.515495539-bleu=0.6774-hidden_dim=256-input_dim=100-epoch=1-batch_size=100-batch_id=[501-[of]-1979]-lr=0.0050'
fusion_model_path = './models_saved/time-[2019-03-10-01-36-12]-info=[pre-trained_fusion_model-20per]-loss=0.365494132-bleu=0.7406-hidden_dim=256-input_dim=100-epoch=1-batch_size=100-batch_id=[501-[of]-1979]-lr=0.0050'

split_model_path = './models_saved/time-[2019-03-10-05-52-44]-info=[pre-trained_split_model-20per]-loss=0.467645884-bleu=0.7270-hidden_dim=256-input_dim=100-epoch=2-batch_size=100-batch_id=[501-[of]-1979]-lr=0.0050'
fusion_model_path = './models_saved/time-[2019-03-10-05-52-48]-info=[pre-trained_fusion_model-20per]-loss=0.327692717-bleu=0.7558-hidden_dim=256-input_dim=100-epoch=2-batch_size=100-batch_id=[501-[of]-1979]-lr=0.0050'

split_model_path = './models_saved/time-[2019-03-10-13-23-10]-info=[pre-trained_split_model-20per]-loss=0.454687029-bleu=0.7130-hidden_dim=256-input_dim=100-epoch=4-batch_size=100-batch_id=[1-[of]-1979]-lr=0.0050'
fusion_model_path = './models_saved/time-[2019-03-10-13-23-11]-info=[pre-trained_fusion_model-20per]-loss=0.346116364-bleu=0.7466-hidden_dim=256-input_dim=100-epoch=4-batch_size=100-batch_id=[1-[of]-1979]-lr=0.0050'

split_model_path = './models_saved/time-[2019-03-16-23-39-39]-info=[split_model-semi]-total_loss=-0.002986051-rec_loss=0.017885875-lm_rewards=0.0102-bleu=0.7734-bleu_bs=0.6282-hidden_dim=256-input_dim=100-epoch=0-batch_size=17-batch_id=[1201-[of]-46585]-lr=0.0050-loss_ratio=0.4500'
fusion_model_path = './models_saved/time-[2019-03-16-23-39-39]-info=[fusion_model-semi]'

pre_train = torch.load(split_model_path, map_location='cpu')
split_model.load_state_dict(pre_train)
pre_train = torch.load(fusion_model_path, map_location='cpu')
fusion_model.load_state_dict(pre_train)

if use_cuda:
    split_model = split_model.cuda()
    fusion_model = fusion_model.cuda()
    
split_optimizer = optim.Adam(filter(lambda p: p.requires_grad, split_model.parameters()), lr=lr)
fusion_optimizer = optim.Adam(filter(lambda p: p.requires_grad, fusion_model.parameters()), lr=lr)

# set_model_grad(fusion_model, False)

In [ ]:
batch_size=17
split_train_set_size=int(len(split_train_set_inputs)/1)
epochs=10000
train_bleu_mean=-1
train_bleu_max=-1
topk=6
loss_ratio=0.45

sup_bsize=35
dataset_times = int(split_train_set_size/len(split_train_set_inputs_supervised))

#batch_size=35, topk=3  or  batch_size=17, topk=6 or  
start_time = time.time()

def model_train(epoch, batch_size, train_set_size):
    batch_id = 0
    valid_bleu = 0
    for start_idx in range(0, train_set_size-batch_size+1, batch_size):
        if batch_id<=1199 and epoch==0:
            batch_id+=1
            continue
#         now = int(round(time.time()*1000))
#         time_stamp = time.strftime(' --->  starting time-[%Y-%m-%d-%H-%M-%S]-',time.localtime(now/1000))
#         print(time_stamp)
        
        #supervised learning
        if batch_id%2==0:
            set_model_grad(split_model, True)
            set_model_grad(fusion_model, False)
            split_optimizer.zero_grad()#clear  
            sup_idx = (batch_id*sup_bsize)%(len(split_train_set_inputs_supervised)-1-sup_bsize)
            split_loss, predicts = split_model.forward(torch.LongTensor(split_train_set_inputs_supervised[sup_idx:sup_idx+sup_bsize]), 
                                         torch.LongTensor(split_train_set_input_lens_supervised[sup_idx:sup_idx+sup_bsize]), 
                                         labels=torch.LongTensor(split_train_set_labels_supervised[sup_idx:sup_idx+sup_bsize]), 
                                         is_train=1, teaching_rate=1)
            split_loss=torch.mean(split_loss)
            split_loss.backward()#retain_graph=True)
            split_optimizer.step()

        if batch_id%2==1:
            set_model_grad(fusion_model, True)
            set_model_grad(split_model, False)
            fusion_optimizer.zero_grad()#clear
            sup_idx = (batch_id*sup_bsize)%(len(split_train_set_inputs_supervised)-1-sup_bsize)
            fusion_loss, predicts = fusion_model.forward(torch.LongTensor(fusion_train_set_inputs_supervised[sup_idx:sup_idx+sup_bsize]), 
                                         torch.LongTensor(fusion_train_set_input_lens_supervised[sup_idx:sup_idx+sup_bsize]), 
                                         labels=torch.LongTensor(fusion_train_set_labels_supervised[sup_idx:sup_idx+sup_bsize]), 
                                         is_train=1, teaching_rate=1)
            fusion_loss = torch.mean(fusion_loss)
            fusion_loss.backward()#retain_graph=True)
            fusion_optimizer.step()
        
        
        #unsupervised learning
        if batch_id%2==0:
#             a=time.time()
            end_idx = start_idx + batch_size
            split_optimizer.zero_grad()#clear
            total_loss, reconstruct_loss, rm_rewards, lm_rewards=split_model.train_using_reward(inputs=torch.LongTensor(split_train_set_inputs[start_idx:end_idx]), 
                                   input_lens=torch.LongTensor(split_train_set_input_lens[start_idx:end_idx]), 
                                   reconstruct_labels=torch.LongTensor(duplicate_reconstruct_labels(fusion_pseudo_train_set_labels[start_idx:end_idx],topk)), 
                                   reconstruct_model=fusion_model, 
                                   language_model=language_model, 
                                   topk=topk, loss_ratio=loss_ratio)
            reconstruct_loss = torch.mean(reconstruct_loss)
            total_loss.backward()#retain_graph=True)
            split_optimizer.step()
#             print('split: all time: ', time.time()-a)
        if batch_id%2==1: 
#             a=time.time()
            end_idx = start_idx + batch_size
            fusion_optimizer.zero_grad()#clear
            total_loss, reconstruct_loss, rm_rewards, lm_rewards=split_model.train_using_reward(inputs=torch.LongTensor(split_train_set_inputs[start_idx:end_idx]), 
                                   input_lens=torch.LongTensor(split_train_set_input_lens[start_idx:end_idx]), 
                                   reconstruct_labels=torch.LongTensor(duplicate_reconstruct_labels(fusion_pseudo_train_set_labels[start_idx:end_idx],topk)), 
                                   reconstruct_model=fusion_model, 
                                   language_model=language_model, 
                                   topk=topk, loss_ratio=loss_ratio)
            reconstruct_loss = loss_ratio*torch.mean(reconstruct_loss)
            reconstruct_loss.backward()#retain_graph=True)
            fusion_optimizer.step()
#             print('fusion: all time: ', time.time()-a)
        #update batch_id
        batch_id+=1
        #timestamp
#         now = int(round(time.time()*1000))
#         time_stamp = time.strftime('time-[%Y-%m-%d-%H-%M-%S]-',time.localtime(now/1000))
#         print(time_stamp)

        torch.cuda.empty_cache()
        #
        if batch_id%20==1:
            split_model.eval()
            fusion_model.eval()
            set_model_grad(split_model, False)
            set_model_grad(fusion_model, False)
            sample_num = 5
            rand_idx = random.randint(0, train_set_size-sample_num-1)
            
            print('--------split model training sampling display--------')
            #teaching forcing
            loss_, predicts = split_model.forward(torch.LongTensor(split_train_set_inputs[rand_idx:rand_idx+sample_num]), 
                                             torch.LongTensor(split_train_set_input_lens[rand_idx:rand_idx+sample_num]), 
                                             labels=torch.LongTensor(split_pseudo_train_set_labels[rand_idx:rand_idx+sample_num]), 
                                             is_train=1, teaching_rate=1)
            del loss_
            
            predicts = batch_tokens_remove_eos(predicts, vocab)
            labels = batch_tokens_remove_eos(split_pseudo_train_set_labels[rand_idx:rand_idx+sample_num], vocab)
            
            predicts = batch_tokens2words(predicts, vocab)
            labels = batch_tokens2words(labels, vocab)
            
            predicts_sents = batch_words2sentence(predicts)
            labels_sents = batch_words2sentence(labels)
            
            for (predict_sent, label_sent) in zip(predicts_sents, labels_sents):
                print(' 1----> ', predict_sent)
                print(' 2----> ', label_sent)
                print('\n')
            
            now = int(round(time.time()*1000))
            time_stamp = time.strftime('time-[%Y-%m-%d-%H-%M-%S]-',time.localtime(now/1000))
            info_stamp = 'info=[{:s}]-total_loss={:2.9f}-rec_loss={:2.9f}-lm_rewards={:5.4f}-hidden_dim={:n}-input_dim={:n}-epoch={:n}-batch_size={:n}-batch_id=[{:n}-[of]-{:n}]-lr={:1.4f}'.format(
                              'split_model', total_loss.data[0], reconstruct_loss.data[0], lm_rewards, 
                            hidden_dim, input_dim, epoch, batch_size, batch_id, int(train_set_size/batch_size), lr)
            print(time_stamp, info_stamp)
            
            if batch_id%40==1:
                #ground truth
#                 rand_idx=random.randint(0, len(split_valid_set_inputs)-batch_size-1-1)
                rand_idx=2333
                loss_, predicts = split_model.forward(torch.LongTensor(split_valid_set_inputs[rand_idx:rand_idx+batch_size]), 
                                                 torch.LongTensor(split_valid_set_input_lens[rand_idx:rand_idx+batch_size]), 
                                                 labels=torch.LongTensor(split_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size]), 
                                                 is_train=1, teaching_rate=1)
                del loss_
#                 predicts = batch_tokens_remove_eos(predicts, vocab)
#                 labels = batch_tokens_remove_eos(split_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size], vocab)
                
#                 bleu_scores = batch_tokens_bleu(references=labels, candidates=predicts, smooth_epsilon=0.001)
                #split version
                bleu_scores = batch_tokens_bleu_split_version(references=split_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size], 
                                                              candidates=predicts, smooth_epsilon=0.001, vocab=vocab)

                valid_bleu = 0
                for x in bleu_scores:
                    valid_bleu+=x
                valid_bleu/=len(bleu_scores)
                
                #beam search
                dec_seqs, log_probs = split_model.dec.decode_topk_seqs(split_model.enc, 
                                                                       inputs=torch.LongTensor(split_valid_set_inputs[rand_idx:rand_idx+batch_size]), 
                                                                         input_lens=torch.LongTensor(split_valid_set_input_lens[rand_idx:rand_idx+batch_size]),
                                                                         topk=topk)
                predicts = []
                for ii in range(len(dec_seqs)):
                    if ii%topk==0:
                        predicts.append(dec_seqs[ii])
               
                bleu_scores = batch_tokens_bleu_split_version(references = split_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size],
                                                             candidates = predicts,
                                                             smooth_epsilon=0.001,
                                                             vocab=vocab)
                valid_bleu_beam_search=0
                for x in bleu_scores:
                    valid_bleu_beam_search+=x
                valid_bleu_beam_search/=len(bleu_scores)


                info_stamp = 'info=[{:s}]-total_loss={:2.9f}-rec_loss={:2.9f}-lm_rewards={:5.4f}-bleu={:1.4f}-bleu_bs={:1.4f}-hidden_dim={:n}-input_dim={:n}-epoch={:n}-batch_size={:n}-batch_id=[{:n}-[of]-{:n}]-lr={:1.4f}-loss_ratio={:1.4f}'.format(
                              'split_model-semi', total_loss.data[0], reconstruct_loss.data[0], lm_rewards, valid_bleu, valid_bleu_beam_search, 
                            hidden_dim, input_dim, epoch, batch_size, batch_id, int(train_set_size/batch_size), lr, loss_ratio)
                
                print(info_stamp, valid_bleu, valid_bleu_beam_search)
                
                now = int(round(time.time()*1000))
                time_stamp = time.strftime('time-[%Y-%m-%d-%H-%M-%S]-',time.localtime(now/1000))
                torch.save(split_model.state_dict(), ''.join(['./models_saved/', time_stamp, info_stamp]))
                torch.save(fusion_model.state_dict(), ''.join(['./models_saved/', time_stamp, 'info=[fusion_model-semi]']))
            set_model_grad(split_model, True)
            set_model_grad(fusion_model, True)
            split_model.train()
            fusion_model.train()
            torch.cuda.empty_cache()
for epoch in range(epochs):
    model_train(epoch, batch_size, split_train_set_size)
    
print('running time: %.2f mins'%((time.time()-start_time)/60))

In [ ]:
stop

In [ ]:
sample_num=2
topk=20

predicts, log_probs=split_model.dec.decode_topk_seqs(split_model.enc, inputs=torch.LongTensor(split_train_set_inputs[0:sample_num]), 
                             input_lens=torch.LongTensor(split_train_set_input_lens[0:sample_num]), 
                             topk=topk)

predicts = batch_tokens_remove_eos(predicts, vocab)
labels = batch_tokens_remove_eos(split_pseudo_train_set_labels[0:sample_num], vocab)

predicts = batch_tokens2words(predicts, vocab)
labels = batch_tokens2words(labels, vocab)

predicts_sents = batch_words2sentence(predicts)
labels_sents = batch_words2sentence(labels)

for idx, sent in enumerate(predicts_sents):
    print(' 1----> ', sent)
    if idx%topk==(topk-1):
        print(' 2----> ', labels_sents[int(idx/topk)])
        print('\n')

In [ ]:
# copy_thres=1.0
# split_loss, predicts = split_model.forward(torch.LongTensor(split_train_set_inputs[0:sample_num]), 
#                                      torch.LongTensor(split_train_set_input_lens[0:sample_num]), 
#                                      labels=torch.LongTensor(split_pseudo_train_set_labels[0:sample_num]), 
#                                      is_train=1, teaching_rate=1)

# predicts = batch_tokens_remove_eos(predicts, vocab)
# labels = batch_tokens_remove_eos(split_pseudo_train_set_labels[0:sample_num], vocab)

# predicts = batch_tokens2words(predicts, vocab)
# labels = batch_tokens2words(labels, vocab)

# predicts_sents = batch_words2sentence(predicts)
# labels_sents = batch_words2sentence(labels)

# for (predict_sent, label_sent) in zip(predicts_sents, labels_sents):
#     print(' 1----> ', predict_sent)
#     print(' 2----> ', label_sent)
#     print('\n')

In [ ]:
stop